In [19]:
# Importing the necessary packages
!pip install tensorflow
!pip install keras

import tensorflow as tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


# Allocate the initial learning rate, Epoch Number and batch size
INIT_LR = 1e-5
# INIT_LR = 20
EPOCHS = 30
BS = 50

DIRECTORY = "/content/drive/MyDrive/United_dataset"
CATEGORIES = ["hat", "withouthat"]

# Upload the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images

print("[INFO] loading images...")

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

# Perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

#  Augmentation generator construction:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# Loading the MobileNetV2 network and freezing the head FC layer sets

baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# HEAD of the model construction which will be placed on top of the base model(BM)

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Glue the head FC model on top of the base model 
#(the resulted model will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# Loop over all layers in the base model and freeze them so they will not
# be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# Compiling the model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

#Checkpointing
chechpoint = tensorflow.keras.callbacks.ModelCheckpoint(
    'v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)
# Train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[chechpoint])

# Make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# For each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# Show formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# Save the model to disk
print("[INFO] saving hat detector model...")
model.save("hat_detector.model", save_format="h5")

# Plot the training loss and accuracy summaries
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

from keras.utils.vis_utils import plot_model
plot_model(model,show_shapes=True)

[INFO] loading images...
[INFO] compiling model...
[INFO] training head...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
86/86 [==============================] - ETA: 0s - loss: 0.8926 - accuracy: 0.4890

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


86/86 [==============================] - 190s 2s/step - loss: 0.8926 - accuracy: 0.4890 - val_loss: 0.7083 - val_accuracy: 0.5373
Epoch 2/30
86/86 [==============================] - 190s 2s/step - loss: 0.7036 - accuracy: 0.6202 - val_loss: 0.5840 - val_accuracy: 0.7539
Epoch 3/30
86/86 [==============================] - 187s 2s/step - loss: 0.6048 - accuracy: 0.7193 - val_loss: 0.4946 - val_accuracy: 0.8673
Epoch 4/30
86/86 [==============================] - 188s 2s/step - loss: 0.5324 - accuracy: 0.7832 - val_loss: 0.4273 - val_accuracy: 0.9078
Epoch 5/30
86/86 [==============================] - 192s 2s/step - loss: 0.4665 - accuracy: 0.8315 - val_loss: 0.3762 - val_accuracy: 0.9189
Epoch 6/30
86/86 [==============================] - 190s 2s/step - loss: 0.4261 - accuracy: 0.8532 - val_loss: 0.3347 - val_accuracy: 0.9253
